In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Loading the Galileo E1 PRNs in hexadecimal format with a pandas dataframe
There exist two different kind of PRN codes for Galileo E1: E1B and E1C.
They are contained, in hexadecimal format, in the files called repectively "E1B - Gal.txt" and "E1C - Gal.txt"

First we start by considering the E1B PRNs:

In [2]:
filename = "E1B - Gal.txt"

# defining the name of the column of the dataframe called e1b
columns = ["Hexadecimal"]

#reading the file
e1b = pd.read_table(filename, sep='\n', header=None, names=columns)

# deleting the first 15 characters (not part of the code)
e1b['Hexadecimal'] = [el[16::] for el in e1b['Hexadecimal']]

# creating a new column with the codes indexes and placing it as first column
e1b['Code index'] = np.arange(1,51,1)
column_list = ["Code index", "Hexadecimal"]
e1b = e1b.reindex(columns=column_list)

# printing the first lines of the dataframe
e1b.head()

,Code index,Hexadecimal
0,1,F5D710130573541B9DBD4FD9E9B20A0D59D144C54BC793...
1,2,96B856A629F581D1344FEF597835FE60434625D077ECF0...
2,3,E57DE19A3E4A8C122FCB1DD6584B3D2DAE364D800F9C5A...
3,4,C0FC4C72A12023BA7093C86775DF3D2F42C7CEDE616876...
4,5,EA9596357B343DFC31D5875CC0E94117A33651472E476D...


In [3]:
# last code (just to verify if the loading operation is correct)
print(e1b['Hexadecimal'][49])

97051FC67ACA30E8AEE73D3A8CF38BB13524D4E0EBD9BE68398C7C16227CABB1D0B0A0ABE7B6384ABA02905BA0C3C7363599D059C7B4C99DB165CD14FA12FA7912449CA7DD5E346D8010C85A757382270DAD15BA3CE36A76EF55F81A1E80BF366B37FE3A88EC722028C25E234E624040450A99CD808F942568AA7133981D72E7F2928894670AD5399482DF1B90E7E64062F830B736C79C30F36281495C76699CD48404673FA334F042F9E0E67DD7F3853BF71ABEAF6A9A5546855E840CE42B224D8F6490C6CE5FC02EBAF4FFC390107058F54CD635D4A7F2878099C1EF495750E6921BE2F39AD808C4210F287319F811A254CEF8CF153FC50AB2F3D694A530949E5F578D075DB96DDCF2BB90ED3DE09D9CA8E08662FD8982741DE1CE0A6B64C3D3D5004B5C04B2B0DFD976A20FACC94D1762D41EE03B40D2CF367612812EF4CC41D1BFE9CEB51AE3A22AF1BE7B85A057D3048D0E73FA0FDAF1119EFD76F0A41BE63128B22D64A5553E9549D411483BBCA1483EF30CF6A6D317AD2C7973EFA6D4C1121F703D2F48FCDA3177AD450D75D2A28D2C244AEA13F0E60AEED8ACBAB444D400DF5E280DB799B2D9A984DF1E2567D39D1DE58EF78CA6B4D8BC172B07DCB02D156CA96EEFAC69E556CFCE0AAB617C7FBB8C34871C1D35E74B7BD307D3F2E424C7A9AD676A1A69E0FE735EA50887A1DFAE6CA2

In [4]:
# simple conversion from hexadecimal to binary
def hex2bin(h):
    integer = int(h, 16)
    binary = bin(integer)[2::]     #discarding the first two elements (i.e. 0b, not needed)
    return binary
    
# creation of a new column of the dataframe with the binary representation of each PRN
e1b['Binary'] = e1b['Hexadecimal'].apply(hex2bin)
e1b

,Code index,Hexadecimal,Binary
0,1,F5D710130573541B9DBD4FD9E9B20A0D59D144C54BC793...,1111010111010111000100000001001100000101011100...
1,2,96B856A629F581D1344FEF597835FE60434625D077ECF0...,1001011010111000010101101010011000101001111101...
2,3,E57DE19A3E4A8C122FCB1DD6584B3D2DAE364D800F9C5A...,1110010101111101111000011001101000111110010010...
3,4,C0FC4C72A12023BA7093C86775DF3D2F42C7CEDE616876...,1100000011111100010011000111001010100001001000...
4,5,EA9596357B343DFC31D5875CC0E94117A33651472E476D...,1110101010010101100101100011010101111011001101...
5,6,90E92279CD4F60D98F6E8FCB3E9263DB60FAB146A835AA...,1001000011101001001000100111100111001101010011...
6,7,A91F570102961D62CA6CB55144AFCCEAF3910F3336DCB0...,1010100100011111010101110000000100000010100101...
7,8,C6E00978E351164532EEA256ECBE0D4F8FCE02A276BD19...,1100011011100000000010010111100011100011010100...
8,9,821BBB3FB91E50253A9E71AC379ED57AEF394C2CC59587...,1000001000011011101110110011111110111001000111...
9,10,92A0DEABA9875D4AFAF99A24C1D5F10EBBE6DEF9CAE5B0...,1001001010100000110111101010101110101001100001...


In [5]:
print(e1b['Binary'][49])    # to verify if the conversion was successful, we print the last code in binary:

# since the last code starts with 9 7 ... in hexadecimal, the binary representation should start with 
# 1001 0111 ...
# and since it ends with ... 2 0, the binary representation should end with
# ... 0010 0000

1001011100000101000111111100011001111010110010100011000011101000101011101110011100111101001110101000110011110011100010111011000100110101001001001101010011100000111010111101100110111110011010000011100110001100011111000001011000100010011111001010101110110001110100001011000010100000101010111110011110110110001110000100101010111010000000101001000001011011101000001100001111000111001101100011010110011001110100000101100111000111101101001100100110011101101100010110010111001101000101001111101000010010111110100111100100010010010001001001110010100111110111010101111000110100011011011000000000010000110010000101101001110101011100111000001000100111000011011010110100010101101110100011110011100011011010100111011011101111010101011111100000011010000111101000000010111111001101100110101100110111111111100011101010001000111011000111001000100000001010001100001001011110001000110100111001100010010000000100000001000101000010101001100111001101100000001000111110010100001001010110100010101010011100010011001110011000

Now we do the same thing for the E1C PRNs, the procedure is exactly the same.
The hexadecimal representation of each PRN is stored in a dataframe called e1c.

In [6]:
filename = "E1C - Gal.txt"
e1c = pd.read_table(filename, sep='\n', header=None, names=columns)

e1c['Hexadecimal'] = [el[16::] for el in e1c['Hexadecimal']]

e1c['Code index'] = np.arange(1,51,1)
column_list = ["Code index", "Hexadecimal"]
e1c = e1c.reindex(columns=column_list)

e1c.head()

,Code index,Hexadecimal
0,1,B39340CA1C817D81EF4FAE4E95BF3504A7709089FB4856...
1,2,A64F94BB47BD4033C76D4924305907EC1F618B43C7535F...
2,3,FD1F6976002C39C87187C44E3D224ED4DF0B6775010594...
3,4,EE38BAF6F61704B01509B5210A0534E4702F93190C392E...
4,5,CD37D0FB0043D03444A939E93676B9DAF5F2D19A2615E3...


In [7]:
e1c['Binary'] = e1c['Hexadecimal'].apply(hex2bin)
e1c

,Code index,Hexadecimal,Binary
0,1,B39340CA1C817D81EF4FAE4E95BF3504A7709089FB4856...,1011001110010011010000001100101000011100100000...
1,2,A64F94BB47BD4033C76D4924305907EC1F618B43C7535F...,1010011001001111100101001011101101000111101111...
2,3,FD1F6976002C39C87187C44E3D224ED4DF0B6775010594...,1111110100011111011010010111011000000000001011...
3,4,EE38BAF6F61704B01509B5210A0534E4702F93190C392E...,1110111000111000101110101111011011110110000101...
4,5,CD37D0FB0043D03444A939E93676B9DAF5F2D19A2615E3...,1100110100110111110100001111101100000000010000...
5,6,CAA02DD19DB9C721EB35AB7D64B8A387796427242698A4...,1100101010100000001011011101000110011101101110...
6,7,FB227530F82BD527E648619E532D7646A5ABBD15DB91A6...,1111101100100010011101010011000011111000001010...
7,8,9E5DA18A19514CCC849E9697AE4BD1B317BB34927D0461...,1001111001011101101000011000101000011001010100...
8,9,8589F8396F5B1C54CAF2B17D4C152CEF347E66EC7903C8...,1000010110001001111110000011100101101111010110...
9,10,A3E17A4CAD2ABE76E32D18501899F8D60D293BB1AC3ADB...,1010001111100001011110100100110010101101001010...


In [8]:
# print(e1c['Binary'][0])    # useful to verify if the conversion was successful
# print(e1c['Binary'][49])    # useful to verify if the conversion was successful